
# The importance of space
Agent based models are useful when the aggregate system behavior emerges out of local interactions amongst the agents. In the model of the evolution of cooperation, we created a set of agents and let all agents play against all other agents. Basically, we pretended as if all our agents were perfectly mixed. In practice, however, it is much more common that agents only interact with some, but not all, other agents. For example, in models of epidemiology, social interactions are a key factors. Thus, interactions are dependend on your social network. In other situations, our behavior might be based on what we see around us. Phenomena like fashion are at least partly driven by seeing what others are doing and mimicking this behavior. The same is true for many animals. Flocking dynamics as exhibited by starling, or shoaling behavior in fish, can be explained by the animal looking at its neirest neighbors and staying within a given distance of them. In agent based models, anything that structures the interaction amongst agents is typically called a space. This space can be a 2d or 3d space with euclidian distances (as in models of flocking and shoaling), it can also be a grid structure (as we will show below), or it can be a network structure. 

MESA comes with several spaces that we can readily use. These are

* **SingleGrid;** an 'excel-like' space with each agent occopying a single grid cell
* **MultiGrid;** like grid, but with more than one agent per grid cell
* **HexGrid;** like grid, but on a hexagonal grid (*e.g.*, the board game Catan) thus changing who your neighbours are
* **ConinuousSpace;** a 2d continous space were agents can occupy any coordinate
* **NetworkGrid;** a network structure were one or more agents occupy a given node.

A key concern when using a none-networked space, is to think carefull about what happens at the edges of the space. In a basic implementation, agents in for example the top left corner has only 2 neighbors, while an agent in the middle has four neighbors. This can give rise to artifacts in the results. Basically, the dynamics at the edges are different from the behavior further away from the edges. It is therefore quite common to use a torus, or donut, shape for the space. In this way, there is no longer any edge and artifacts are thus removed.


# The emergence of cooperation in space
The documentation of MESA on the different spaces is quite limited. Therefore, this assignment is largely a tutorial continuing on the evolution of cooperation. 

We make the following changes to the model

* Each agent gets a position, which is an x,y coordinate indicating the grid cell the agent occupies.
* The model has a grid, with an agent of random class. We initialize the model with equal probabilities for each type of class.
* All agents play against their neighbors. On a grid, neighborhood can be defined in various ways. For example, a Von Neumann neighborhood contains the four cells that share a border with the central cell. A Moore neighborhood with distance one contains 8 cells by also considering the diagonal. Below, we use a neighborhood distance of 1, and we do include diagonal neighbors. So we set Moore to True. Feel free to experiment with this model by setting it to False, 
* The evolutionary dynamic, after all agents having played, is that each agent compares its scores to its neighbors. It will adopt whichever strategy within its neighborhood performed best.
* Next to using a SingleGrid from MESA, we also use a DataCollector to handle collecting statistics.

Below, I discuss in more detail the code containing the most important modifications.


In [1]:
from collections import deque, Counter, defaultdict
from enum import Enum
from itertools import combinations
from math import floor

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from mesa import Model, Agent
from mesa.space import SingleGrid
from mesa.datacollection import DataCollector

#Geen idee wat dit precies doet; leuk.
class Move(Enum):
    COOPERATE = 1
    DEFECT = 2


class AxelrodAgent(Agent):
    """An Abstract class from which all strategies have to be derived
    
    
    Attributes
    ----------
    points : int
    pos : tuple
    
    """
    #Initialiseert een agent met twee atributen, die positie pos, hetgeen een tulip met een x en y waarde dient te zijn tussen de 0 en 20 vlgnsm mij. en natuurlijk de andere atribuut is het aantal punten.
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, model)
        self.points = 0
        self.pos = pos

#Afhankelijk welke strategie we willen komt hier een andere invulling te staan voor de subclass
    def step(self):
        '''
        This function defines the move and any logic for deciding
        on the move goes here.
        
        Returns
        -------
        Move.COOPERATE or Move.DEFECT
        
        '''
        #Mocht er iets fout gaan dan een error. logisch toch.
        raise NotImplemetedError

#Recieve payoff is iets dat elke agent in het model moet kunnen krijgen dus daarom staat het hier. de attribuut wordt self.points wordt geupdate. Maar dat hangt natuurlijk af van de payoff, die kan ook 0 zijn. Je moet natuurlijk wel winnen en dat hangt af van de voorgaande methode step.
    def receive_payoff(self, payoff, my_move, opponent_move):
        '''receive payoff and the two moves resulting in this payoff
        
        Parameters
        ----------
        payoff : int
        my_move : {Move.COOPERATE, Move.DEFECT}
        opponements_move : {Move.COOPERATE, Move.DEFECT}
        
        '''
        self.points += payoff
        
        #Deze functie; reset wordt pas ingezet als het hele spelletje voorbij is en alles weer op nul gezet wordt. Wat dat reset inhoud is soms anders en daarom is het gespecificeerd in de aparte agent sub classes.
    def reset(self):
        '''
        This function is called after playing N iterations against
        another player.
        '''
        raise NotImplementedError

#Dit is de eerste agent strategie.
class TitForTat(AxelrodAgent):
    """This class defines the following strategy: play nice, unless, 
    in the previous move, the other player betrayed you."""
    
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, pos, model)
        #Voordat het model gelopen heeft moet de tit voor tat al wel weten wat die de eerste keer moet doen. Voor de rest hang het altijd af van de opponent_last_move. Daarom zetten we die hier nu gewoon al neer. Daarna wordt die in de step method overwritten.
        self.opponent_last_move = Move.COOPERATE
    
    def step(self):
        #De agent wordt straks in een model gestopt en daar wordt opgeslagen wat de opponnent_last move was; die returned deze agent nu.
        return self.opponent_last_move
    
    def receive_payoff(self, payoff, my_move, opponent_move):

        #Kijk hier is de recieve payoff beter gespecifieerd. De super() staat er voor dat een method in de bovenstaande klass wordt ingevuld,          namelijk de receiv_payoff moet namelijk weten wat de payoff moet zijn. Nu hangt die af van de my_move en de opponent_move:
        # In het model zijn de payoffs gedefineert:
        # payoff_a, payoff_b = self.payoff_matrix[(move_a, move_b)]
        # agent_a.receive_payoff(payoff_a, move_a, move_b)
        # agent_b.receive_payoff(payoff_b, move_b, move_a)

        #Dit geeft een tulip waarde voor a en b, die heet payoff en wordt toegevoegd aan de betreffende agent; het is een beetje stom dat er hier verwezen moet worden voor de simpele operator =+ payoff, maar ja.

        super().receive_payoff(payoff, my_move, opponent_move)
        self.opponent_last_move = opponent_move
        
    def reset(self):
        self.opponent_last_move = Move.COOPERATE


class ContriteTitForTat(AxelrodAgent):
    """This class defines the following strategy:  play nice, unless, 
    in the previous two moves, the other player betrayed you.""" 
    
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, pos, model)
        self.opponent_last_two_moves = deque([Move.COOPERATE, Move.COOPERATE], maxlen=2)
    
    def step(self):
        if (self.opponent_last_two_moves[0] == Move.DEFECT) and\
           (self.opponent_last_two_moves[1] == Move.DEFECT):
            return Move.DEFECT
        else:
            return Move.COOPERATE
    
    def receive_payoff(self, payoff, my_move, opponent_move):
        super().receive_payoff(payoff, my_move, opponent_move)
        self.opponent_last_two_moves.append(opponent_move)
        
    def reset(self):
        self.opponent_last_two_moves = deque([Move.COOPERATE, Move.COOPERATE], maxlen=2)        


#############################
# Models from Assignment 1 #
#############################

class Defector(AxelrodAgent):
    def step(self):
        return Move.DEFECT
        
    def reset(self):
        pass

class Cooperator(AxelrodAgent):
    def step(self):
        return Move.COOPERATE
    
    def reset(self):
        pass
    
class GrimTrigger(AxelrodAgent):
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, pos, model)
        self.betrayed = False

        
    def step(self):
        if self.betrayed:
            return Move.DEFECT
        else:
            return Move.COOPERATE
        
    def receive_payoff(self, payoff, my_move, opponent_move):
        super().receive_payoff(payoff, my_move, opponent_move)
        if not self.betrayed:
            if opponent_move == Move.DEFECT:
                self.betrayed = True
        
    def reset(self):
        self.betrayed = False
        
class Pavlov(AxelrodAgent):
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, pos, model)
        self.opponent_last_move = deque([Move.COOPERATE], maxlen=1)
        self.my_move_last_move = deque([Move.COOPERATE], maxlen=1)
    
    def step(self):
        if self.opponent_last_move[0] == self.my_move_last_move[0]:
            return self.my_move_last_move[0]
        else:
            if self.my_move_last_move[0] == Move.COOPERATE:
                return Move.DEFECT
            else:
                return Move.COOPERATE
    
    def receive_payoff(self, payoff, my_move, opponent_move):
        super().receive_payoff(payoff, my_move, opponent_move)
        self.opponent_last_move.append(opponent_move)
        self.my_move_last_move.append(opponent_move)
        
    def reset(self):
        self.opponent_last_move = deque([Move.COOPERATE], maxlen=1)
        self.my_move_last_move = deque([Move.COOPERATE], maxlen=1)

class Random(AxelrodAgent):
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, pos, model)
    
    def step(self):
        if self.random.getrandbits(1):
            return Move.DEFECT
        else:
            return Move.COOPERATE
        
    def reset(self):
        pass

class NoisySpatialEvolutionaryAxelrodModel(Model):
    #okey here we define the space in which te agents have to operate
    #First initialize the model, giving it the parameters of noise_level, number of rounds N and the space of the space in 2D. 
    #height and width. From the superclass Model we just need the seed which is set at seed, conventiantly.

    def __init__(self, N, noise_level=0.01, seed=None,
                 height=20, width=20):
        super().__init__(seed=seed)

        #The noise level is defined to compare to the random generator later on
        self.noise_level = noise_level
        self.num_iterations = N

        #For the agents to be stored in a sort of list is created but it is not a list it is a set. What is the difference, not much but apperaently better for the memory.
        self.agents = set()

        #Then in the model the payoff is defined. First a dictionary is created and for all 4 combinations of moves, which act as keys in the dictionairy, values are set in the forms of tulips. so tulips act as keys for tulip values in a dictionary! to be used in a set which is a sort of list. I don't really understand it either.

        self.payoff_matrix = {}
        self.payoff_matrix[(Move.COOPERATE, Move.COOPERATE)] = (2, 2)
        self.payoff_matrix[(Move.COOPERATE, Move.DEFECT)] = (0, 3)
        self.payoff_matrix[(Move.DEFECT, Move.COOPERATE)] = (3, 0)
        self.payoff_matrix[(Move.DEFECT, Move.DEFECT)] = (1, 1)        
        

        #Okey now we set op the space of grid. Which is apperently also an attribute to the model class. Luckily there are predifined grids we can pick from. We choose you, SingleGrid!
        self.grid = SingleGrid(width, height, torus=True)
        
        #Okey now we estbalish which strategies are available, they are stored in a list apperantly. By refereing to AxelrodAgent.__subclasses__() it will returnn a list with all the different subclasses == strategies
        
        strategies = AxelrodAgent.__subclasses__()
    #Apperantly we need to know how many strategies there are so lets extrect the lenght of the list == number of strategies
        num_strategies = len(strategies)
    #Now we create a new attribute for this model called agent_id which apperently has to point to the number 0 for now.
        self.agent_id = 0
        
    #Okey, this becomes somewhat complicated; appereently there is a attribute (hence the self.) called grid.coord.iter() This is amethod in the Singlegrid method from the MESA library. self.grid was defined earlier as SingleGrid. This method apperently iteratas over all the cells in the grid. Here 20 by 20 = 400 cells.
   
        for cell in self.grid.coord_iter():
            #Each cell apperently becomes a tulip with the values, _, x, y. _ means that it is empty (for now)
            _, x, y = cell
            #Furthermore the posistion of each cell gets stored in the pos attribute for now. But apperantly no self. needs to be put in front of it.
            pos = (x, y)
            # The self.agent which was 0 gets now a 1 yeah!
            self.agent_id += 1
# Now we again create a variable called strategy_index to create a random integer.
            strategy_index = int(floor(self.random.random()*num_strategies))
# Now we create another variable called agent which uses the above variable strategy_index whole integer number to acquire from the long set of strategies a certein agent_id, posistion and self
            agent = strategies[strategy_index](self.agent_id, pos, self)
# This agent is then placed at the grid at the pos.
#Okey, so we store the posistion both in the agent and in the model; which makes sense.
            self.grid.position_agent(agent, (x, y))
#The agent along its information of id and pos are stored in the set. which is a sort of list
            self.agents.add(agent)
# Now we create a new attribute which by calling a method from the MESA library calledd Datacollector. I suppose it collects data.      
        self.datacollector = DataCollector(model_reporters={klass.__name__:klass.__name__
                                                            for klass in strategies})
#Since this is a for loop the cells get filled, but I dont yet know if all cells are filled? Furthermore than the ID of all agent is from 1 to 20 which is a bit dull.


#Okey lets create a new method; count! Yes apperaently we want to count stuff.
    def count_agent_types(self):
        counter = Counter()
        #For all agents stored in the self.agent set we create a dictionary with a name and how many we get from them.
        for agent in self.agents:
            #This means that we look at the agents in the set, we look at the class from which the agent is an instance(__class__.), then we look at the name of the class (__name_). this way we keep track of the amount of similar strategies going about in the model. 
            counter[agent.__class__.__name__] += 1
#The counter gives to elements, since it is a dictionary; it gives the key, nameley the name here, and the amount of strategies going abour, for example 4 times the TitforTat. 
        for k,v in counter.items():
            #The k is the key and the v is the value, 4!
            #Setettribute does something very very special, adds attributes to a certain instance of a class.
            #here the self is the agent. The agents has now the instance attributes of agent_id and its posistion. But we apperantly want to add to the instance also the information of its strategie name, k and the amount of similar strategies there are presetn v. I don't see the point of the letter attribute yet however.
               setattr(self, k, v)    

    def step(self):
        #so the model calls on the stored dictionary of the counter first
        '''Advance the model by one step.'''
        self.count_agent_types()
    
        self.datacollector.collect(self)
        #Agent_a wordt hier toegezen aan de eerste agent, met pos
        for (agent_a, x, y) in self.grid.coord_iter():
            #Agent_b wordt hier toegewzen met een speciale functie.
            for agent_b in self.grid.neighbor_iter((x,y), moore=True):
                for _ in range(self.num_iterations):
                    #Dit blijft zo doorgaan voor het aantal iteraties gedefenieerd in deze model class.
                    #de agent is die nu agent_a roept nu zijn eigen step functie aan. De move die eruit komt wordt toegewezen
                    move_a = agent_a.step()
                    move_b = agent_b.step()

                    #insert noise in movement
                    if self.random.random() < self.noise_level:
                        if move_a == Move.COOPERATE:
                            move_a = Move.DEFECT
                        else:
                            move_a = Move.COOPERATE
                    if self.random.random() < self.noise_level:
                        if move_b == Move.COOPERATE:
                            move_b = Move.DEFECT
                        else:
                            move_b = Move.COOPERATE
                    #de moves als tulip worden als input gebruikt voor de payoff,method die kijkt naar de dictionary met bijbehorende uikomsten tulip
                    payoff_a, payoff_b = self.payoff_matrix[(move_a, move_b)]
    #Voor de agenten wordt de method aangeroepen die de payoff hierboven gebruik als input en wordt opgeteld in de superclass van de agents; hier Axelrod Agent; helemaal bovenin
                    agent_a.receive_payoff(payoff_a, move_a, move_b)
                    agent_b.receive_payoff(payoff_b, move_b, move_a)
                agent_a.reset()
                agent_b.reset()

        # evolution
        # tricky, we need to determine for each grid cell
        # is a change needed, if so, log position, agent, and type to change to
        agents_to_change = []
        for agent_a in self.agents:
            neighborhood = self.grid.iter_neighbors(agent_a.pos, moore=True,
                                                    include_center=True)
            neighborhood = ([n for n in neighborhood])
            neighborhood.sort(key=lambda x:x.points, reverse=True)
            best_strategy = neighborhood[0].__class__
            # if best type of strategy in neighborhood is
            # different from strategy type of agent, we need
            # to change our strategy
            if not isinstance(agent_a, best_strategy):
                agents_to_change.append((agent_a, best_strategy))

        for entry in agents_to_change:
            agent, klass = entry
            self.agents.remove(agent)
            self.grid.remove_agent(agent)
            
            pos = agent.pos
            self.agent_id += 1
            
            new_agent = klass(self.agent_id, pos, self)
            self.grid.position_agent(new_agent, pos)
            self.agents.add(new_agent)



In the `__init__`, we now instantiate a SingleGrid, with a specified width and height. We set the kwarg torus to True indicating we are using a donut shape grid to avoid edge effects. Next, we fill this grid with random agents of the different types. This can be implemented in various ways. What I do here is using a list with the different classes (*i.e.*, types of strategies). By drawing a random number from a unit interval, multiplying it with the lenght of the list of classes and flooring the resulting number to an integer, I now have a random index into this list with the different classes. Next, I can get the class from the list and instantiate the agent object.

Some minor points with instantiating the agents. First, we give the agent a position, called pos, this is a default attribute assumed by MESA. We also still need a unique ID for the agent, we do this with a simple counter (`self.agent_id`). `self.grid.coord_iter` is a method on the grid. It returns an iterator over the cells in the grid. This iterator returns the agent occupying the cell and the x and y coordinate. Since the first item is `null` because we are filling the grid, we can ignore this. We do this by using the underscore variable name (`_`). This is a python convention. 

Once we have instantiated the agent, we place the agent in the grid and add it to our collection of agents. If you look in more detail at the model class, you will see that I use a set for agents, rather than a list. The reason for this is that we are going to remove agents in the evolutionary phase. Removing agents from a list is memory and compute intensive, while it is computationally easy and cheap when we have a set. 

```python
self.grid = SingleGrid(width, height, torus=True)

strategies = AxelrodAgent.__subclasses__()
num_strategies = len(strategies)
self.agent_id = 0

for cell in self.grid.coord_iter():
    _, x, y = cell
    pos = (x, y)

    self.agent_id += 1

    strategy_index = int(floor(self.random.random()*num_strategies))

    agent = strategies[strategy_index](self.agent_id, pos, self)
    self.grid.position_agent(agent, (x, y))
    self.agents.add(agent)
```

We also use a DataCollector. This is a default class provided by MESA that can be used for keeping track of relevant statistics. It can store both model level variables as well as agent level variables. Here we are only using model level variables (i.e. attributes of the model). Specifically, we are going to have an attribute on the model for each type of agent strategy (i.e. classes). This attribute is the current count of agents in the grid of the specific type. To implement this, we need to do several things.

1. initialize a data collector instance
2. at every step update the current count of agents of each strategy
3. collect the current counts with the data collector.

For step 1, we set a DataCollector as an attribute. This datacollector needs to know the names of the attributes on the model it needs to collect. So we pass a dict as kwarg to model_reporters. This dict has as key the name by which the variable will be known in the DataCollector. As value, I pass the name of the attribute on the model, but it can also be a function or method which returns a number. Note th at the ``klass`` misspelling is deliberate. The word ``class`` is protected in Python, so you cannot use it as a variable name. It is common practice to use ``klass`` instead in the rare cases were you are having variable refering to a specific class.

```python
self.datacollector = DataCollector(model_reporters={klass.__name__:klass.__name__
                                                    for klass in strategies})
```

For step 2, we need to count at every step the number of agents per strategy type. To help keep track of this, we define a new method, `count_agent_types`. The main magic is the use of `setattr` which is a standard python function for setting attributes to a particular value on a given object. This reason for writing our code this way is that we automatically adapt our attributes to the classes of agents we have, rather than hardcoding the agent classes as attributes on our model. If we now add new classes of agents, we don't need to change the model code itself. There is also a ``getattr`` function, which is used by for example the DataCollector to get the values for the specified attribute names. 

```python
def count_agent_types(self):
    counter = Counter()
    for agent in self.agents:
        counter[agent.__class__.__name__] += 1

    for k,v in counter.items():
        setattr(self, k, v)    

```

For step 3, we modify the first part of the ``step`` method. We first count the types of agents and next collect this data with the datacollector.
```python
self.count_agent_types()
self.datacollector.collect(self)
```

The remainder of the ``step`` method has also been changed quite substantially. First, We have to change against whom each agent is playing. We do this by iterating over all agents in the model. Next, we use the grid to give us the neighbors of a given agent. By setting the kwarg ``moore`` to ``True``, we indicate that we include also our diagonal neighbors. Next, we play as we did before in the noisy version of the Axelrod model.

```python
for agent_a in self.agents:
    for agent_b in self.grid.neighbor_iter(agent_a.pos, moore=True):
        for _ in range(self.num_iterations):
```

Second, we have to add the evolutionary dynamic. This is a bit tricky. First, we loop again over all agents in the model. We check its neighbors and see which strategy performed best. If this is of a different type (``not isinstance(agent_a, best_strategy)``, we add it to a list of agents that needs to be changed and the type of agent to which it needs to be changed. Once we know all agents that need to be changed, we can make this change. 

Making the change is quite straighforward. We remove the agent from the set of agents (`self.agents`) and from the grid. Next we get the position of the agent, we increment our unique ID counter, and create a new agent. This new agent is than added to the grid and to the set of agents. 

```python
# evolution
agents_to_change = []
for agent_a in self.agents:
    neighborhood = self.grid.iter_neighbors(agent_a.pos, moore=True,
                                            include_center=True)
    neighborhood = ([n for n in neighborhood])
    neighborhood.sort(key=lambda x:x.points, reverse=True)
    best_strategy = neighborhood[0].__class__
    # if best type of strategy in neighborhood is
    # different from strategy type of agent, we need
    # to change our strategy
    if not isinstance(agent_a, best_strategy):
        agents_to_change.append((agent_a, best_strategy))

for entry in agents_to_change:
    agent, klass = entry
    self.agents.remove(agent)
    self.grid.remove_agent(agent)

    pos = agent.pos
    self.agent_id += 1

    new_agent = klass(self.agent_id, pos, self)
    self.grid.position_agent(new_agent, pos)
    self.agents.add(new_agent)

```

## Assignment 1
Can you explain why we need to first loop over all agents before we are changing a given agent to a different strategy?

In [2]:
#The first loop goes over all agents first because we need to get the data of the games; it is about the averages of a strategie not only the first round. After a first round of 1 strategie you can't decide yet. 

## Assignment 2
Add all agents classes (i.e., strategie) from the previous assignment to this model. Note that you might have to update the ``__init__`` method to reflect the new pos keyword argument and attribute. 

In [ ]:
#__init__(self, unique_id, pos, model)
#super().__init__(unique_id, pos, model)
#Are changed


## Assignment 3
Run the model for 50 steps, and with 200 rounds of the iterated game. Use the defaults for all other keyword arguments.

Plot the results. 

*hint: the DataCollector can return the statistics it has collected as a dataframe, which in turn you can plot directly.*

In [7]:
model = NoisySpatialEvolutionaryAxelrodModel(10, 200)

for _ in range(50):
    model.step()



AttributeError: 'NoisySpatialEvolutionaryAxelrodModel' object has no attribute 'scores'

In [28]:
print(DataCollector())
fig, ax = plt.subplots()
#The extraction of the agents and the values from the datacollector is too opaque for me to do. the documentation is too limited
for k, v in DataCollector(values):
    
    ax.plot(v, label=k.__name__)
ax.legend()
plt.show()

NameError: name 'values' is not defined

This new model is quite a bit noisier than previously. We have a random initialization of the grid and depending on the initial neighborhood, different evolutionary dynamics can happen. On top, we have the noise in game play, and the random agent. 


## Assignment 4
Let's explore the model for 10 replications. Run the model 10 times, with 200 rounds of the iterated prisoners dilemma. Run each model for fifty steps. Plot the results for each run. 

1. Can you say anything generalizable about the behavioral dynamics of the model?
2. What do you find striking in the results and why?
3. If you compare the results for this spatially explicit version of the Emergence of Cooperation with the none spatially explicit version, what are the most important differences in dynamics. Can you explain why adding local interactions results in these changes? 